In [1]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
client = MongoClient('localhost', 27017)
# client = MongoClient('mongodb://localhost:27017/')
db = client.db_cdiscount

In [2]:
import io
import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
from skimage.data import imread   # or, whatever image library you prefer
import multiprocessing as mp      # will come in handy due to the size of the data

In [3]:
import numpy as np

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import keras
K.set_image_dim_ordering('th')

from keras import regularizers

Using TensorFlow backend.


In [25]:
first = 0
last = 1000
batch = 400
num_batch = int(last/batch)
# last = 82


In [26]:
epochs = 50
batch_size = 16
num_classes = db.cat_encode.count()

In [27]:
def simple_model(epochs=epochs, num_classes=num_classes, lrate=0.01, momentum=0.9):
    
    # Convolutional Layer
    model = Sequential()
    # Convolutional Layer
    model.add(Conv2D(40, (3,3), input_shape = (180,180,3), activation='relu'))

    # Pooling Layer
    model.add(MaxPooling2D(pool_size=(2, 1)))
    
    # Fully conected Layer
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))


    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

#     model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [28]:
def deeper_model(epochs=epochs, num_classes=num_classes, lrate=0.01, momentum=0.9):
    # Convolutional Layer
    model = Sequential()
    # Convolutional Layer
    model.add(Conv2D(10, (3,3), input_shape = (180,180,3), activation='relu',
                     kernel_regularizer= regularizers.l2(0.5)))
#     model.add(Dropout(0.2))
    model.add(Conv2D(13, 3, activation='relu', padding='same'))
#     model.add(Dropout(0.2))
    model.add(Conv2D(14, 3, activation='relu', padding='same'))
    model.add(Dropout(0.2))
    
    # Pooling Layer
    model.add(MaxPooling2D(pool_size=(2, 1)))
    
    
    model.add(Conv2D(20, 3, activation='relu', padding='same'))
#     model.add(Dropout(0.2))
    model.add(Conv2D(26, 3, activation='relu', padding='same'))
#     model.add(Dropout(0.2))
    model.add(Conv2D(20, 3, activation='relu', padding='same'))
    model.add(Dropout(0.2))
    model.add(Conv2D(15, 3, activation='relu', padding='same'))
#     model.add(Dropout(0.2))
    model.add(Conv2D(10, 3, activation='relu', padding='same'))
#     model.add(Dropout(0.2))
    model.add(Conv2D(10, 3, activation='relu', padding='same'))
    
    model.add(MaxPooling2D(pool_size=(2, 1)))
    
    model.add(Conv2D(30, 3, activation='relu', padding='same'))
    model.add(Dropout(0.2))
    model.add(Conv2D(25, 3, activation='relu', padding='same'))
    
    model.add(MaxPooling2D(pool_size=(2, 1)))
    
    model.add(Conv2D(30, 3, activation='relu', padding='same'))
    model.add(Dropout(0.2))
    model.add(Conv2D(30, 3, activation='relu', padding='same'))
    model.add(Conv2D(10, 3, activation='relu', padding='same'))
    model.add(Dropout(0.2))
    
    model.add(MaxPooling2D(pool_size=(2, 1)))
    
    
    # Fully conected Layer
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))


    lrate = lrate
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=momentum, decay=decay, nesterov=False)

#     model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
    

In [29]:
def deeper_model2(epochs=epochs, num_classes=num_classes, lrate=0.01, momentum=0.9, l2=1):
    # Convolutional Layer
    model = Sequential()
    # Convolutional Layer
    model.add(Conv2D(10, (3,3), input_shape = (180,180,3), activation='relu'))
    #     model.add(Dropout(0.2))
    model.add(Conv2D(54, 3, activation='relu', padding='same'))
#     model.add(Dropout(0.2))

    model.add(MaxPooling2D(pool_size=(4, 1)))
    model.add(Conv2D(128, 3, activation='relu', padding='same'))
#     model.add(Dropout(0.2))

    model.add(MaxPooling2D(pool_size=(4, 1)))
    model.add(Conv2D(256, 3, activation='relu', padding='same'))
#     model.add(Dropout(0.2))  

    model.add(MaxPooling2D(pool_size=(3, 1)))
    # Fully conected Layer
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))


    lrate = lrate
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=momentum, decay=decay, nesterov=False)

    #     model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [42]:
def deeper_model3(epochs=epochs, num_classes=num_classes, lrate=0.01, momentum=0.9, l2=0.78):
    # Convolutional Layer
    model = Sequential()
    # Convolutional Layer
    model.add(Conv2D(10, (3,3), input_shape = (180,180,3), activation='relu', 
                    kernel_regularizer= regularizers.l2(l2)))
    model.add(Dropout(0.1))
    model.add(Conv2D(54, 3, activation='relu', padding='same'))
    model.add(Dropout(0.2))

    model.add(MaxPooling2D(pool_size=(4, 1)))
    model.add(Conv2D(128, 3, activation='relu', padding='same'))
    model.add(Dropout(0.3))

    model.add(MaxPooling2D(pool_size=(4, 1)))
    model.add(Conv2D(256, 3, activation='relu', padding='same'))
    model.add(Dropout(0.4))  

    model.add(MaxPooling2D(pool_size=(3, 1)))
    # Fully conected Layer
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))


    lrate = lrate
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=momentum, decay=decay, nesterov=False)

    #     model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [43]:
# model =simple_model(epochs=epochs, num_classes=num_classes)
model =deeper_model3(epochs=epochs, num_classes=num_classes)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 40, 178, 1)        64840     
_________________________________________________________________
dropout_21 (Dropout)         (None, 40, 178, 1)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 80, 178, 1)        12880     
_________________________________________________________________
dropout_22 (Dropout)         (None, 80, 178, 1)        0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 80, 44, 1)         0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 128, 44, 1)        41088     
_________________________________________________________________
dropout_23 (Dropout)         (None, 128, 44, 1)        0         
__________

In [44]:
import time

In [ ]:
start_time = time.time()
print('start code')
counter = 0
row_count = 0
for j in range(0,num_batch-1):
# for j in range(0,1):
    batch_time = time.time()
    a1 = int(list(np.linspace(first,last,num_batch))[j])
    an = int(list(np.linspace(first,last,num_batch))[j+1])
    
    lst_batch = []
    
    print(a1,an)
    
    cur = db.train.find({})[a1:an]
    
    m= an-a1
        
    i = 0
#     pic_array
    while (cur.alive):
        idx = cur.next()
        dic = {}
        
        category_id =idx['category_id'] 
    #     print(category_id)

        cat = db.cat_encode.find_one({ "cat" : (category_id)}, {"cat" : 1.0, "_id" : 0})['cat']
    #     print(cat)
        
        picture = np.float32(imread(io.BytesIO(idx['imgs'][0]['picture']))/255.0)
        picture = picture.reshape(180,180,3)
        
        dic['picture'] = picture
        
        encode = db.cat_encode.find_one({ "cat" : (category_id)}, {"encode" : 1.0, "_id" : 0})['encode']
    #     print(encode)
        dic['encode'] = encode
        
        lst_batch.append(dic)
        
        i+=1
    X_batch = np.array([lst['picture'] for lst in lst_batch])
    Y_batch = np.array([lst['encode'] for lst in lst_batch])
    print('X batch size: ', X_batch.shape)
    print('Y batch size: ', Y_batch.shape)
    
    del(lst_batch)
        
    model.fit(X_batch, Y_batch, batch_size=batch_size, epochs=epochs, validation_split=0.3)
    
#     print(X_batch.shape[0])
    row_count = row_count + X_batch.shape[0]
    counter = counter + X_batch.shape[0]
    
    if(row_count >= 100000):
        print('row count: ',row_count)
        counter = counter + row_count
        model_name = 'model_' + str(counter) +'.h5'
        print(model_name)
#         model.model.save('E://kaggle//Cdiscount//model//'+model_name)
        print('Model saved')
        row_count = 0
    
    print("--- %s seconds ---" % (time.time() - batch_time))
    print('rows completed: ', counter)
    
print("--- %s seconds ---" % (time.time() - start_time))
# model.model.save('E://kaggle//Cdiscount//model//final_model.h5')

start code
0 1000
X batch size:  (1000, 180, 180, 3)
Y batch size:  (1000, 5270)
Train on 700 samples, validate on 300 samples
Epoch 1/50
700/700 [==============================] - 16s - loss: 14.0114 - acc: 0.3914 - val_loss: 8.8137 - val_acc: 0.4033
Epoch 2/50
700/700 [==============================] - 9s - loss: 5.1219 - acc: 0.4129 - val_loss: 5.7093 - val_acc: 0.4033
Epoch 3/50
700/700 [==============================] - 9s - loss: 3.9464 - acc: 0.4129 - val_loss: 5.2158 - val_acc: 0.4033
Epoch 4/50
700/700 [==============================] - 8s - loss: 3.3294 - acc: 0.4171 - val_loss: 4.7729 - val_acc: 0.4100
Epoch 5/50
700/700 [==============================] - 9s - loss: 2.9876 - acc: 0.4357 - val_loss: 4.7813 - val_acc: 0.4200
Epoch 6/50
700/700 [==============================] - 9s - loss: 2.7019 - acc: 0.4457 - val_loss: 4.6751 - val_acc: 0.4367
Epoch 7/50
700/700 [==============================] - 9s - loss: 2.4377 - acc: 0.4686 - val_loss: 4.7865 - val_acc: 0.4467
Epoch 8/50

# Get Prediction on learned model

In [ ]:
lst_prediction = []
cur = db.train.find({}).limit(last)

while (cur.alive):
    idx = cur.next()
    dic = {}
    
    ### this variable is stored for model performance and will not used in test set###
    dic['Actual'] = idx['category_id']
    
    picture = np.float32(imread(io.BytesIO(idx['imgs'][0]['picture']))/255.0)
#     plt.imshow(picture)
#     plt.show()
    picture = picture.reshape(1,180,180,3)
#     yFit = model.predict_on_batch(picture)
    
    yFit = model.predict_on_batch(picture)

#     print('Actual: ', idx['category_id'])
#     print()
    # print(yFit.shape)
    # print('yFit: ', yFit[0][0])
#     print('max Argumet: ', yFit.argmax())

    arg_max_idx = float(yFit.argmax())

    tmp = db.argmax_encode.find_one({'arg_max':arg_max_idx})
#     print('arg_max_db: ', tmp['arg_max'])
#     print('Predicted_category_id: ', tmp['cat'])

    
    dic['_id'] =idx['_id']
    dic['category_id'] = tmp['cat']
#     print()
    
#     print(dic)
    lst_prediction.append(dic)

# lst_prediction

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(lst_prediction, columns=['_id','category_id', 'Actual'])
df.head()

In [ ]:
df['compare']= df['category_id'] == df['Actual']
print(df[df['compare']==True].head())
print('Accuracy: ', len(df[df['compare']==True]['_id'])/last)